<a href="https://colab.research.google.com/github/fmarol/Assignments/blob/master/ST446_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment 1

Username: fmarol

All images can be found in the Image directory also uploaded in the GitHub repository.

We start by setting up the bucket and cluster. For part 1 & 2 we need to use the author-large.txt file, so we download this from the url link and place it in our bucket. 

The steps taken to do this are as follows:

Set up bucket:

```gsutil mb gs://fmarol-bucket```

Create cluster:

```gcloud beta dataproc clusters create fmarol-cluster --optional-components=ANACONDA,JUPYTER     --image-version=1.3     --enable-component-gateway  --bucket fmarol-bucket --project st446-267513```

Into SSH:

```gcloud beta compute --project "st446-267513" ssh --zone "europe-west2-a" "fmarol-cluster-m"```

Download text file from the URL, and then copy into the bucket:

```wget http://webdam.inria.fr/Jorge/files/author-large.txt```

```gsutil cp author-large.txt gs://fmarol-bucket```
gsutil cp author-large.txt gs://fmarol-bucket
The code ran to do this can be seen in Image 1 and 2.

### Part 1
First we download the data from the text file and transform it so that it has the form of a dataset: 

In [0]:
data_from_file = sc.\
    textFile("gs://fmarol-bucket/author-large.txt", 4)

In [0]:
data_from_file.take(5)

[u'Jurgen Annevelink\tModern Database Systems\tObject SQL - A Language for the Design and Implementation of Object Databases.\t1995',
 u'Rafiul Ahad\tModern Database Systems\tObject SQL - A Language for the Design and Implementation of Object Databases.\t1995',
 u'Amelia Carlson\tModern Database Systems\tObject SQL - A Language for the Design and Implementation of Object Databases.\t1995',
 u'Daniel H. Fishman\tModern Database Systems\tObject SQL - A Language for the Design and Implementation of Object Databases.\t1995',
 u'Michael L. Heytens\tModern Database Systems\tObject SQL - A Language for the Design and Implementation of Object Databases.\t1995']

In [0]:
import numpy as np
data_from_file_conv = data_from_file.map(lambda row: np.array(row.strip().split("\t")))
data_from_file_conv.take(5)

[array([u'Jurgen Annevelink', u'Modern Database Systems',
        u'Object SQL - A Language for the Design and Implementation of Object Databases.',
        u'1995'], dtype='<U78'),
 array([u'Rafiul Ahad', u'Modern Database Systems',
        u'Object SQL - A Language for the Design and Implementation of Object Databases.',
        u'1995'], dtype='<U78'),
 array([u'Amelia Carlson', u'Modern Database Systems',
        u'Object SQL - A Language for the Design and Implementation of Object Databases.',
        u'1995'], dtype='<U78'),
 array([u'Daniel H. Fishman', u'Modern Database Systems',
        u'Object SQL - A Language for the Design and Implementation of Object Databases.',
        u'1995'], dtype='<U78'),
 array([u'Michael L. Heytens', u'Modern Database Systems',
        u'Object SQL - A Language for the Design and Implementation of Object Databases.',
        u'1995'], dtype='<U78')]

Next, we take just the Name and Title columns, saved as ```book_authors```.

Then, we join the dataset on itself by the column Title and this gives us all the combinations of pairs of authors, including repeats:

In [0]:
book_authors = data_from_file_conv.map(lambda row: (row[2], row[0]))

book_authors_join = book_authors.join(book_authors).filter(lambda row: row[1][0]<row[1][1])
book_authors_join.take(5)

[(u'Impact of the Processing Methods on the Performance of the X-ray Film Screen Combinations.',
  (u'Cao Hou-de', u'Wang Yong-ming')),
 (u'Fusing multiple systems into a compact lattice index for chinese spoken term detection.',
  (u'Peng Yu', u'Sha Meng')),
 (u'Fusing multiple systems into a compact lattice index for chinese spoken term detection.',
  (u'Jia Liu', u'Sha Meng')),
 (u'Fusing multiple systems into a compact lattice index for chinese spoken term detection.',
  (u'Jia Liu', u'Peng Yu')),
 (u'Fusing multiple systems into a compact lattice index for chinese spoken term detection.',
  (u'Frank Seide', u'Sha Meng'))]

Before counting the number of titles per pair, we check to see if the rows are unique. If they are not then we might count pairs multiple times for one title:

In [0]:
book_authors_join.map(lambda row: ((row[0], row[1]), 1)).reduceByKey(lambda x, y: x+y).filter(lambda row: row[1] != 1).take(10)

[((u'Introduction.', (u'James Ohene-Djan', u'Robert Logcher')), 2),
 ((u'Adaptive Video on Demand.', (u'Amir Herzberg', u'Sudhanshu Aggarwal')),
  4),
 ((u'Minitrack Introduction.', (u'Jrg M. Haake', u'Judith Gebauer')), 2),
 ((u'Minitrack Introduction.', (u'Robert O. Briggs', u'Samuli Pekkola')), 2),
 ((u'Minitrack Introduction.',
   (u'Christopher P. Holland', u'Mark N. Frolick')),
  2),
 ((u'A Semantic Data Grid for Satellite Mission Quality Analysis.',
   (u'Manuel Snchez-Gestido', u'scar Corcho')),
  4),
 ((u'3.', (u'Gershon Elber', u'Yuefei Zhu')), 2),
 ((u'Minitrack Introduction.', (u'Pirkko Walden', u'Scott McCoy')), 4),
 ((u'BLOG: Probabilistic Models with Unknown Objects.',
   (u'Andrey Kolobov', u'Brian Milch')),
  4),
 ((u'Minitrack Introduction.', (u'Kenneth J. Trimmer', u'Marios Koufaris')),
  2)]

As one can see from the output above, the rows are not unique and some pair-title combinations appear more than once.

To avoid counting them multiple times, we group by the pair-title combinations, before mapping the pairs to values of one:

In [0]:
count = book_authors_join.map(lambda row: ((row[0], row[1]), 1)).groupByKey().mapValues(list).map(lambda row: (row[0][1], 1))
count.take(5)

[((u'Don DeSota', u'Sameh Sharkawi'), 1),
 ((u'Xiao-Yuan Jing', u'Yong-Chuan Zhang'), 1),
 ((u'Guy-Ren Perrin', u'Wolfgang Karl'), 1),
 ((u'J. W. Park', u'Y. S. Moon'), 1),
 ((u'Shuichi Sakai', u'Toshitsugu Yuba'), 1)]

Finally we reduce the author pairs, summing up the mapped values and then sorting the sums in descending order to give the top 10 author pairs:

In [0]:
count.reduceByKey(lambda x, y: x+y).sortBy(lambda row: row[1], ascending = False).take(10)

[((u'Irith Pomeranz', u'Sudhakar M. Reddy'), 246),
 ((u'Amr El Abbadi', u'Divyakant Agrawal'), 161),
 ((u'Makoto Takizawa', u'Tomoya Enokido'), 137),
 ((u'Didier Dubois', u'Henri Prade'), 122),
 ((u'Elizabeth Chang', u'Tharam S. Dillon'), 115),
 ((u'Mary Jane Irwin', u'Narayanan Vijaykrishnan'), 107),
 ((u'Mahmut T. Kandemir', u'Mary Jane Irwin'), 100),
 ((u'Chun Chen', u'Jiajun Bu'), 99),
 ((u'Shojiro Nishio', u'Takahiro Hara'), 96),
 ((u'Filip De Turck', u'Piet Demeester'), 90)]

### Part 2
Now we do the same analysis using PySpark:

In [0]:
filename = 'gs://fmarol-bucket/author-large.txt'

from pyspark.sql.types import *

schema = StructType([
    StructField("author", StringType(), True),    
    StructField("journal", StringType(), True),
    StructField("title", StringType(), True),
    StructField("year", LongType(), True)
])

author_large = spark.read.csv(filename, 
                    header='false', schema=schema, sep='\t')
author_large.createOrReplaceTempView("author_large")

The Spark SQL code does something very similar as the operations with the RDD.

First a temporary table ```all_combos``` is created by joining the dataset on itself by the title column. From this, the author pair and total number of distinct titles per author pair are extracted and then ordered descending by total number.

The same results are seen:

In [0]:
spark.sql("""with all_combos as (select CONCAT(a2.author, ', ', a1.author) author_pair, a1.title
        from author_large a1 join author_large a2 on a1.title=a2.title where a1.author>a2.author)
        select author_pair, count(distinct title) count from all_combos group by author_pair order by count desc""").show(10, truncate = False)

+----------------------------------------+-----+
|author_pair                             |count|
+----------------------------------------+-----+
|Irith Pomeranz, Sudhakar M. Reddy       |246  |
|Amr El Abbadi, Divyakant Agrawal        |161  |
|Makoto Takizawa, Tomoya Enokido         |137  |
|Didier Dubois, Henri Prade              |122  |
|Elizabeth Chang, Tharam S. Dillon       |115  |
|Mary Jane Irwin, Narayanan Vijaykrishnan|107  |
|Mahmut T. Kandemir, Mary Jane Irwin     |100  |
|Chun Chen, Jiajun Bu                    |99   |
|Shojiro Nishio, Takahiro Hara           |96   |
|Filip De Turck, Piet Demeester          |90   |
+----------------------------------------+-----+
only showing top 10 rows



### Part 3

For this part we use a different dataset, Yelp, which is contained in a zipfile on DropBox. To in order to load the JSON file into a database in Hive, a SerDe must be used. The SerDe is contained in a JAR file which can be downloaded from this website: https://mvnrepository.com/artifact/org.apache.hive.hcatalog/hive-hcatalog-core/0.13.0.

First both the Yelp JSON file and JAR file are downloaded to the local drive, and then uploaded to the bucket on GCP:

```gsutil cp 'D:/yelp_academic_dataset_user.json' gs://fmarol-bucket```

```gsutil cp 'C:/Users/loram/OneDrive/Desktop/LSE MSc/ST446/hive-hcatalog-core-3.1.2.jar' gs://fmarol-bucket```

Into SSH

```gcloud beta compute --project "st446-513267" ssh --zone "europe-west2-a" "fmarol-cluster-m"```

The Yelp file and JAR file are then copied onto the .....:

```gsutil cp gs://fmarol-bucket/yelp_academic_dataset_user.json .```

```gsutil cp gs://fmarol-bucket/hive-hcatalog-core-3.1.2.jar .```

Then the files are put on the HDFS server:

hadoop -put yelp_academic_dataset_user.json /

hadoop -put hive-hcatalog-core-3.1.2.jar /

Then we enter Hive:

```hive```

The following code is then ran in order to:
- add the jar file that contains the SerDe for the JSON file

```ADD JAR hive-hcatalog-core-3.1.2.jar;```

- create a database to hold the Yelp dataset in

```CREATE DATABASE IF NOT EXISTS yelp_user_database;```

- create an empty table, with the schema of the Yelp JSON file, and the appropriate SerDe file path

```USE yelp_user_database;```

```
CREATE TABLE IF NOT EXISTS yelp_users (
  user_id STRING,
  name STRING,
  review_count INT,
  yelping_since STRING,
  friends ARRAY<STRING>,
  useful INT,
  funny INT,
  cool INT,
  fans INT,
  elite ARRAY<STRING>,
  verage_stars FLOAT,
  compliment_hot INT,
  compliment_more INT,
  compliment_profile INT,
  compliment_cute INT,
  compliment_list INT,
  compliment_note INT,
  compliment_plain INT,
  compliment_cool INT,
  compliment_funny INT,
  compliment_writer INT,
  compliment_photos INT,
  type STRING ) 
ROW FORMAT SERDE 'org.apache.hive.hcatalog.data.JsonSerDe';
```

- load the JSON file into the empty table

```LOAD DATA LOCAL INPATH 'yelp_academic_dataset_user.json' INTO TABLE yelp_users;```

The above code has been run and outputs can be seen in Images 3 and 4.

Now that the dataset has been prepared, we answer the questions. We enter the database:
```USE yelp_database;```

1. Retrieve the schema for the dataset:

```DESCRIBE yelp_users;```

2. Count the number of rows in the dataset:

```SELECT COUNT(*) FROM yelp_users;```

3. Select top 10 users who have provided the largest number of reviews

First we check to see that there are no repeated user_ids by counting the distinct user_ids and seeing if this is equal to the number of rows.

```SELECT COUNT(DISTINCT user_id) FROM yelp_users;```

Seeing that these are equal, we go on to rank the users by number of reviews:

```SELECT name, review_count FROM yelp_users ORDER BY review_count DESC LIMIT 10;```

All above code and outputs can be seen in Images 5 to 8.